# Quick visualization

## Description

This program does the following creates a Folium map with the selected repeat stations and the magnetic observatories in South America. The presented information is:
- The basic information of each station (name, coordinates, number of occupation, etc)
- The temporal series for each station
- The basic information for each observatory

In [1]:
# Import modules
import mestrado_module as mm
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import geopandas as gpd
from shapely.geometry import Point, Polygon
import folium
from folium import plugins
import seaborn as sns
import branca
from branca.element import Template, MacroElement
import json
import altair as alt

In [2]:
# Magnetic Observatories
sa_obs_folder: Path = Path(mm.path_00_data_manual)
sa_obs_file: Path = Path("models_intermagnet_observatories_south_america.csv")

# Magnetic stations
mag_sts_folder: Path = Path(mm.path_00_data_manual)
mag_sts_file: Path = Path("models_magnetic_stations_brazil.csv")

# Selected repeat stations (folium file)
selected_rs_folder: Path = Path(mm.path_pipeline_07_select_rs_geo_distribution)
selected_rs_file: Path = Path(mm.output_7a_code_selected_rs_folium)

# Selecter repea stations database
selected_rs_database_folder: Path = Path(mm.path_pipeline_07_select_rs_geo_distribution)
selected_rs_database_file: Path = Path(mm.output_7b_code_selected_rs_db)

# Folium Maps
mapa5 = "../02_pipeline/07_select_repeat_stations_geo_distribution/07e_quick_visualization_repeat_stations.html"

## Read data

In [3]:
# Station database
# STATIONS DATA
selected_rs_db = pd.read_csv(selected_rs_database_folder / selected_rs_database_file)

# Check info
#selected_rs_db.info()

In [4]:
# STATIONS DATA
selected_rs = pd.read_csv(selected_rs_folder /selected_rs_file)

# this has to be removed, otherwise folium wont read the geometry correctly later for some reason
# Therefore is better to create the geometry everytime it is needed
selected_rs = selected_rs.drop(columns=["geometry"]) 

# Check info
#selected_rs.info()

In [5]:
# INTERMAGNET OBSERVATORIES IN SOUTH AMERICA
south_america_obs_df = pd.read_csv(sa_obs_folder / sa_obs_file)

# Check df info
#south_america_obs_df.info()

In [6]:
# PRESENT-DAY MAGNETIC STATION
mag_sts_df = pd.read_csv(mag_sts_folder / mag_sts_file)

# Check df info
#mag_sts_df.info()

## Create the geodataframe for the folium map

In [7]:
# Repeat stations data
geometry_selected_rs = [Point(xy) for xy in zip(selected_rs["Lon_dd"], selected_rs["Lat_dd"])]
# Create the geodataframe (use the df_aux dataframe and create the geometry column)
gdf_selected_rs = gpd.GeoDataFrame(selected_rs, geometry = geometry_selected_rs)

# Check it
#gdf_selected_rs.info()

In [8]:
# INTERMAGNET OBSERVATORIES IN SOUTH AMERICA
south_america_obs_geometry = [Point(xy) for xy in zip(south_america_obs_df["Lon_dd"], south_america_obs_df["Lat_dd"])]

# Create the geodataframe
gdf_south_america_obs = gpd.GeoDataFrame(south_america_obs_df, geometry = south_america_obs_geometry)
#gdf_south_america_obs.info()

In [9]:
# PRESENT-DAY MAGNETIC STATION
mag_sts_geometry = [Point(xy) for xy in zip(mag_sts_df["Lon_dd"], mag_sts_df["Lat_dd"])]

# Create the geodataframe
gdf_mag_sts = gpd.GeoDataFrame(mag_sts_df, geometry = mag_sts_geometry)
#gdf_mag_sts.info()

## Plot Folium map: information and time series layers

In [10]:
## DEFINITIONS (distances in km)
#observatory_coverage_radius = 904000 # BASED ON ITALY CASTELLO TESINO
#embracer_coverage_radius = 150000 # germany
#embracer_coverage_radius = 230000 # italy GIB station

# Radius suggested by Katia and Cristiano (800 km, 300 km)
observatory_coverage_radius_m = 800000
repeat_station_coverage_radius_m = 300000
magnetic_station_coverage_radius_m = 500000

# Color definitions
sa_obs_color = "darkblue"
selected_rs_color = "black"
mag_sts_color = "pink"

# Custom icons for the repeat stations and observatories
icon_path_sa_obs = r"../00_data/manual/symbols_square_darkblue.png"
icon_path_rs = r"../00_data/manual/symbols_circle_black.png"
icon_path_mag_sts = r"../00_data/manual/symbols_triangle_pink.png"

In [11]:
# Fuction to create popup with repeat station information
def popup_html(row):
    i = row
    code= gdf_selected_rs['Code'].iloc[i] 
    rs_name= gdf_selected_rs['RS_name'].iloc[i]
    lat = gdf_selected_rs['Lat_dd'].iloc[i] 
    lon= gdf_selected_rs['Lon_dd'].iloc[i] 
    n_ocp = gdf_selected_rs['N_occupations'].iloc[i]                     
    last_ocp = gdf_selected_rs['Time_dy'].iloc[i]
    closest_obs = gdf_selected_rs['Closest_OBS'].iloc[i]

    left_col_color = "#19a7bd"
    right_col_color = "#f2f0d3"
    
    html = """<!DOCTYPE html>
<html>

<head>
<h4 style="margin-bottom:10"; width="200px">{}</h4>""".format(code) + """

</head>
    <table style="height: 126px; width: 350px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Name</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(rs_name) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Latitude (dd)</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(lat) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Longitude (dd)</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(lon) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Total occupations</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(n_ocp) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Last occupation</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(last_ocp) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Nearest Brazilian Observatory</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(closest_obs) + """
</tr>
</tbody>
</table>
</html>
"""
    return html

In [12]:
# Fuction to create popup with repeat station information
def popup_html_observatory(df, row):
    i = row
    code= df['Code'].iloc[i] 
    obs_name= df['Name'].iloc[i]
    lat = df['Lat_dd'].iloc[i] 
    lon = df['Lon_dd'].iloc[i] 
    altitude = df['Altitude_m'].iloc[i]                     
    country = df['Country'].iloc[i]

    left_col_color = "#19a7bd"
    right_col_color = "#f2f0d3"
    
    html_observatory = """<!DOCTYPE html>
<html>

<head>
<h4 style="margin-bottom:10"; width="200px">{}</h4>""".format(code) + """

</head>
    <table style="height: 126px; width: 350px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Name</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(obs_name) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Latitude (dd)</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(lat) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Longitude (dd)</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(lon) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Altitude (m)</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(altitude) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Country</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(country) + """
</tr>
</tbody>
</table>
</html>
"""
    return html_observatory

In [13]:
# FUNCTION
# This function creates a folium marker and the altair charts for a station. The input is the station name.
# To make sure it works

def create_folium_marker_altair(df_main, rs_name):
    
    # Define the database
    db = df_main
    
    # Define parameters
    time = "Time_dy"
    compf = "F_nT"
    comph = "H_nT"
    compx = "X_nT"
    compx_calc = "Calculated_X"
    compy = "Y_nT"
    compy_calc = "Calculated_Y"
    compz = "Z_nT"
    compz_calc = "Calculated_Z"
    compd = "D_dd"
    compi = "I_dd"
    chart_width = 400
    chart_height = 200
    popup_max_width = 800
    popup_width = 700
    popup_height = 700
    icon_path_rs = r"../00_data/manual/symbols_circle_black.png"
    
    # Create df
    df0 = db[db["Code"] == rs_name]
    
    # Get last index of the dataframe
    df0_last_index = df0.last_valid_index()
    
    # Lat lon points
    lat0 = db["Lat_dd"].iloc[df0_last_index]
    lon0 = db["Lon_dd"].iloc[df0_last_index]
    
    # Magnetic Components
    comp_fh = pd.DataFrame(
        {"Time_dy": df0[time],
         "Total Field (nT)": df0[compf],
         "Horizontal field (nT)": df0[comph]
        }
    )
    # Merger data in relation to time axis
    data_fh = comp_fh.melt(time)
    # create an altair chart, then convert to JSON
    chart0 = alt.Chart(data_fh).mark_line(point = True).encode(
        x = time, y = "value", color = "variable").properties(
        width = chart_width,
        height = chart_height,
        title = f"F and H fields at the station {rs_name}").interactive()

    # Magnetic Components
    comp_xyz = pd.DataFrame(
        {"Time_dy": df0[time],
         "X Field (nT)": df0[compx],
         "Calculated X Field (nT)": df0[compx_calc],
         "Y Field (nT)": df0[compy],
         "Calculated Y Field (nT)": df0[compy_calc],
         "Z Field (nT)": df0[compz],
         "Calculated Z field (nT)": df0[compz_calc],
        }
    )
    # Merger data in relation to time axis
    data_xyz = comp_xyz.melt(time)
    # create an altair chart
    chart01 = alt.Chart(data_xyz).mark_line(point=True).encode(
        x = time, y = "value", color = "variable").properties(
        width = chart_width,
        height = chart_height,
        title = f"X Y Z fields at the station {rs_name}").interactive()

    # Magnetic Components
    comp_di = pd.DataFrame(
        {"Time_dy": df0[time],
         "Declination (dd)": df0[compd],
         "Inclination (dd)": df0[compi]
        }
    )
    # Merger data in relation to time x axis
    data_di = comp_di.melt(time)
    # create an altair chart
    chart001 = alt.Chart(data_di).mark_line(point=True).encode(
        x = time, y = "value", color = "variable").properties(
        width = chart_width,
        height = chart_height,
        title = f"D and I fields at the station {rs_name}").interactive()

    # Merge charts to form subplots in Altair
    merged_charts = chart0 & chart01 & chart001

    # Convert merged chart to JSON
    vis0 = merged_charts.to_json()
    
    # Create Folium marker
    marker0 = folium.Marker(
        [lat0, lon0], 
        popup = folium.Popup(max_width = popup_max_width).add_child(folium.VegaLite(vis0, width = popup_width, height = popup_height)),
        icon=folium.features.CustomIcon(icon_path_rs, icon_size=(14, 14)),
        tooltip = rs_name + ", Click me to see the time series")
    
    return marker0


### Create each marker for the time series layer

In [14]:
# List with station names
list_sts = selected_rs_db.Code.unique()
#print(list_sts)

# Use the function to create a folium marker for each station
marker00 = create_folium_marker_altair(selected_rs_db, list_sts[0])
marker01 = create_folium_marker_altair(selected_rs_db, list_sts[1])
marker02 = create_folium_marker_altair(selected_rs_db, list_sts[2])
marker03 = create_folium_marker_altair(selected_rs_db, list_sts[3])
marker04 = create_folium_marker_altair(selected_rs_db, list_sts[4])
marker05 = create_folium_marker_altair(selected_rs_db, list_sts[5])
marker06 = create_folium_marker_altair(selected_rs_db, list_sts[6])
marker07 = create_folium_marker_altair(selected_rs_db, list_sts[7])
marker08 = create_folium_marker_altair(selected_rs_db, list_sts[8])
marker09 = create_folium_marker_altair(selected_rs_db, list_sts[9])
marker10 = create_folium_marker_altair(selected_rs_db, list_sts[10])
marker11 = create_folium_marker_altair(selected_rs_db, list_sts[11])
marker12 = create_folium_marker_altair(selected_rs_db, list_sts[12])
marker13 = create_folium_marker_altair(selected_rs_db, list_sts[13])
marker14 = create_folium_marker_altair(selected_rs_db, list_sts[14])
marker15 = create_folium_marker_altair(selected_rs_db, list_sts[15])
marker16 = create_folium_marker_altair(selected_rs_db, list_sts[16])
marker17 = create_folium_marker_altair(selected_rs_db, list_sts[17])
marker18 = create_folium_marker_altair(selected_rs_db, list_sts[18])
marker19 = create_folium_marker_altair(selected_rs_db, list_sts[19])
marker20 = create_folium_marker_altair(selected_rs_db, list_sts[20])
marker21 = create_folium_marker_altair(selected_rs_db, list_sts[21])
marker22 = create_folium_marker_altair(selected_rs_db, list_sts[22])
marker23 = create_folium_marker_altair(selected_rs_db, list_sts[23])
marker24 = create_folium_marker_altair(selected_rs_db, list_sts[24])
marker25 = create_folium_marker_altair(selected_rs_db, list_sts[25])
marker26 = create_folium_marker_altair(selected_rs_db, list_sts[26])
marker27 = create_folium_marker_altair(selected_rs_db, list_sts[27])
marker28 = create_folium_marker_altair(selected_rs_db, list_sts[28])
marker29 = create_folium_marker_altair(selected_rs_db, list_sts[29])
marker30 = create_folium_marker_altair(selected_rs_db, list_sts[30])
marker31 = create_folium_marker_altair(selected_rs_db, list_sts[31])
marker32 = create_folium_marker_altair(selected_rs_db, list_sts[32])
marker33 = create_folium_marker_altair(selected_rs_db, list_sts[33])
marker34 = create_folium_marker_altair(selected_rs_db, list_sts[34])
marker35 = create_folium_marker_altair(selected_rs_db, list_sts[35])
marker36 = create_folium_marker_altair(selected_rs_db, list_sts[36])
marker37 = create_folium_marker_altair(selected_rs_db, list_sts[37])
marker38 = create_folium_marker_altair(selected_rs_db, list_sts[38])
marker39 = create_folium_marker_altair(selected_rs_db, list_sts[39])
marker40 = create_folium_marker_altair(selected_rs_db, list_sts[40])
marker41 = create_folium_marker_altair(selected_rs_db, list_sts[41])
marker42 = create_folium_marker_altair(selected_rs_db, list_sts[42])
marker43 = create_folium_marker_altair(selected_rs_db, list_sts[43])
marker44 = create_folium_marker_altair(selected_rs_db, list_sts[44])
marker45 = create_folium_marker_altair(selected_rs_db, list_sts[45])
marker46 = create_folium_marker_altair(selected_rs_db, list_sts[46])
marker47 = create_folium_marker_altair(selected_rs_db, list_sts[47])
marker48 = create_folium_marker_altair(selected_rs_db, list_sts[48])
marker49 = create_folium_marker_altair(selected_rs_db, list_sts[49])

In [15]:
# INTERACTIVE MAP: Map of selected repeat stations
# Create the Interactive map using folium

start_coords = [-15, -50]
tile_type0 = "OpenStreetMap"
map5 = folium.Map(location = start_coords, zoom_start = 5, zoom_control = True, scrollWheelZoom = True, dragging = True, tiles = tile_type0 )

# Main group
fg0 = folium.FeatureGroup()    

# Group 1: Selected Repeat Stations
g01 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Repeat Stations, inactive radius', show = True)
for i in range(0,len(gdf_selected_rs)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html,width=510,height=280)
    popup = folium.Popup(folium.Html(html, script=True), max_width=500)
    folium.Marker([gdf_selected_rs['Lat_dd'].iloc[i], gdf_selected_rs['Lon_dd'].iloc[i]],
                  popup = popup, icon = folium.features.CustomIcon(icon_path_rs, icon_size=(14, 14)),
                  tooltip = "Click me to see Repeat Station information, to see time series select it in the layer control button (top right button) the layer Repeat Station, time series").add_to(g01)
    
# Group 2: Selected Repeat Stations
g02 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Repeat Stations, active radius', show = False)
for i in range(0,len(gdf_selected_rs)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html,width=510,height=280)
    popup = folium.Popup(folium.Html(html, script=True), max_width=500)
    folium.Marker([gdf_selected_rs['Lat_dd'].iloc[i], gdf_selected_rs['Lon_dd'].iloc[i]],
                  popup = popup, icon = folium.features.CustomIcon(icon_path_rs, icon_size=(14, 14)),
                  tooltip = "Click me to see Repeat Station information, to see time series select it in the layer control button (top right button) the layer Repeat Station, time series").add_to(g02)
# Add the repeat stations coverage    
for index, location_info in gdf_selected_rs.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = repeat_station_coverage_radius_m,
        color = selected_rs_color #,
        #fill_color = "green"    
    ).add_to(g02)
    

# Group 3: Selected Repeat Stations, time series
g03 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Repeat Stations, time series', show = False)
# Add markers to folium group
marker00.add_to(g03)
marker01.add_to(g03)
marker02.add_to(g03)
marker03.add_to(g03)
marker04.add_to(g03)
marker05.add_to(g03)
marker06.add_to(g03)
marker07.add_to(g03)
marker08.add_to(g03)
marker09.add_to(g03)
marker10.add_to(g03)
marker11.add_to(g03)
marker12.add_to(g03)
marker13.add_to(g03)
marker14.add_to(g03)
marker15.add_to(g03)
marker16.add_to(g03)
marker17.add_to(g03)
marker18.add_to(g03)
marker19.add_to(g03)
marker20.add_to(g03)
marker21.add_to(g03)
marker22.add_to(g03)
marker23.add_to(g03)
marker24.add_to(g03)
marker25.add_to(g03)
marker26.add_to(g03)
marker27.add_to(g03)
marker28.add_to(g03)
marker29.add_to(g03)
marker30.add_to(g03)
marker31.add_to(g03)
marker32.add_to(g03)
marker33.add_to(g03)
marker34.add_to(g03)
marker35.add_to(g03)
marker36.add_to(g03)
marker37.add_to(g03)
marker38.add_to(g03)
marker39.add_to(g03)
marker40.add_to(g03)
marker41.add_to(g03)
marker42.add_to(g03)
marker43.add_to(g03)
marker44.add_to(g03)
marker45.add_to(g03)
marker46.add_to(g03)
marker47.add_to(g03)
marker48.add_to(g03)
marker49.add_to(g03)

 
# Group 4: Magnetic Observatories in South America
g04 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Magnetic Observatories in South America, inactive radius', show = True)
for i in range(0,len(gdf_south_america_obs)):
    html_obs = popup_html_observatory(gdf_south_america_obs, i)
    iframe = branca.element.IFrame(html=html,width=510,height=280)
    popup = folium.Popup(folium.Html(html_obs, script=True), max_width=500)
    folium.Marker([gdf_south_america_obs['Lat_dd'].iloc[i], gdf_south_america_obs['Lon_dd'].iloc[i]],
                  popup=popup,icon=folium.features.CustomIcon(icon_path_sa_obs, icon_size=(14, 14)),
                  tooltip = "Click me to see Observatory information").add_to(g04)
    
# Group 5: Magnetic Observatories in South America
g05 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Magnetic Observatories in South America, active radius', show = False)
for i in range(0,len(gdf_south_america_obs)):
    html_obs = popup_html_observatory(gdf_south_america_obs, i)
    iframe = branca.element.IFrame(html=html,width=510,height=280)
    popup = folium.Popup(folium.Html(html_obs, script=True), max_width=500)
    folium.Marker([gdf_south_america_obs['Lat_dd'].iloc[i], gdf_south_america_obs['Lon_dd'].iloc[i]],
                  popup=popup,icon=folium.features.CustomIcon(icon_path_sa_obs, icon_size=(14, 14)),
                  tooltip = "Click me to see Observatory information").add_to(g05)
# Add the observatory coverage    
for index, location_info in gdf_south_america_obs.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = observatory_coverage_radius_m,
        color = sa_obs_color,
        fill_color = sa_obs_color
    ).add_to(g05)
        

# Add the subgroups to the main map
map5.add_child(fg0)
map5.add_child(g01)
map5.add_child(g02)
map5.add_child(g03)
map5.add_child(g04)
map5.add_child(g05)

# Add other layers to the map
folium.TileLayer('Stamen Terrain').add_to(map5)
folium.TileLayer('Stamen Toner').add_to(map5)
#folium.TileLayer('Stamen Water Color').add_to(map5)
folium.TileLayer('cartodbpositron').add_to(map5)
#folium.TileLayer('cartodbdark_matter').add_to(map5)

# Add the layer control
folium.LayerControl().add_to(map5)

# Add altitude and longitude tool map
lat_lon_vis0 = folium.LatLngPopup()
map5.add_child(lat_lon_vis0)

# Measurement control
measure_control0 = plugins.MeasureControl(position = "topleft", 
                                         active_color = "red", 
                                         completed_color = "red", 
                                         primary_length_unit = "kilometers")
map5.add_child(measure_control0)

# Add the full screen button
fullscreen_button0 = plugins.Fullscreen(position ='topright', title ='Expand me',
                                title_cancel ='Exit me',
                                force_separate_button = True)
map5.add_child(fullscreen_button0)

# Add a mini map
minimap5 = plugins.MiniMap()
map5.add_child(minimap5)

# Draw tools
# export=True exports the drawn shapes as a geojson file
draw0 = plugins.Draw(export=True)
map5.add_child(draw0)


#--------------------------------------------------------------
# ADD LEGEND TO THE MAP
template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Quick visualization: The 50 selected repeat stations</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Legend (draggable!)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:black;opacity:0.7;'></span>Repeat Stations</li>
    <li><span style='background:darkblue;opacity:0.7;'></span>Magnetic Observatories</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)
map5.get_root().add_child(macro)

# Save map
map5.save(mapa5)